# Example Sample Hotel and Flight Booker Agent 

Dis solution go help you book flight tickets and hotel. Di scenario na trip from London Heathrow LHR Feb 20th 2024 go New York JFK and e go return Feb 27th 2025, di flight na economy wit British Airways only. I want make I stay for one Hilton hotel for New York, abeg provide costs for di flight and di hotel.


# Initialize di Azure AI Agent Service and get configuration information from **.env**

### **.env** 

Create one .env file 

**.env** get di connection string of Azure AI Agent Service, di model wey AOAI dey use, and di corresponding Google API Search service API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Your Azure AI Agent Service Model Deployment Name"

[**NOTE**] You go need model wey get 100,000 Rate Limit (Tokens per minute) Rate Limit of 600 (Request per minute)

  You fit get model from Microsoft Foundry - Model and Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Your Azure AI Agent Service Project Connection String"

  You fit get di project connection string for your project overview for AI ​​Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Your SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Your SERPAPI Search Endpoint"

To get the Model Deployment Name and Project Connection String of Azure AI Agent Service, you need create Azure AI Agent Service. E better make you use [this template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) to create am directly （***Note:*** Azure AI Agent Service dey currently set for one limited region. E better make you check [this link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) to set di region)

Agent need to access SERPAPI. E better make you register using [this link](https://serpapi.com/searches). After you don register, you fit get your unique API KEY and ENDPOINT.


# Setup 

To run dis notebook, you go need make sure say you don install di required libraries by running `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Your Semantic Kernel version suppose dey at least 1.27.2.


Load your .env file setting and resources abeg make sure say you don add your keys and setting and don create local .env file.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Log in to Azure

You dey need log in to Azure now. Open terminal come run dis command:

```bash
az login
```

Dis command go make you enter your Azure credentials, so dat the Azure AI Agent service go fit work well.


# Explanation:
Dis na one variable wey dey store di API key wey you go use access SERP (Search Engine Results Page) API service. API key na unique identifier wey dey authenticate request dem wey relate to your account.

Purpose: Di purpose of dis line na to store di API key for one variable so dat e go fit use authenticate request dem to di SERP API service. You go need di API key to fit access di service and perform search dem.
How to Get a SERP API Key: To get one SERP API key, follow dis general step dem for https://serpapi.com (di exact step fit differ depend on di particular SERP API service wey you dey use):

Choose a SERP API Service: E get plenti SERP API services wey dey available, like SerpAPI, Google Custom Search JSON API, and oda ones. Choose di one wey best suit your need.

Sign Up for an Account: Go di website of di chosen SERP API service and sign up for account. You fit need provide some basic information and verify your email address.

Create an API Key: After you don sign up, login to your account and waka go di API section or dashboard. Find option to create or generate new API key.
Copy di API Key go your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Na variable wey dey store di base URL for di SERP API endpoint. Di variable name BASE_URL na convention wey dem dey use to show say dis URL na di starting point to make API requests.
'https://serpapi.com/search':

Dis na di correct URL string wey dem assign to di BASE_URL variable. E mean di endpoint for to perform search queries using di SERP API.

# Purpose:
Di purpose of dis line na to define constant wey hold di base URL for di SERP API. Dis URL go dey use as di starting point to construct API requests to do search operations.

# Usage:
By to define di base URL for one variable, you fit easily reuse am all through your code anytime you need to make requests to di SERP API. Dis one go make your code easier to maintain and e go reduce di risk of errors wey fit happen if you hardcode di URL for many places. Di current example na https://serpapi.com/search?engine=bing wey dey use Bing search API. You fit select different API for https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explanation:

Dis na where your plugin code dey.

Class Definition: `class BookingPlugin`: E define one class wey dem call BookingPlugin wey get methods for booking hotels and flights.

Hotel Booking Method:

- `@kernel_function(description="booking hotel")`: Na decorator wey talk say dis function na kernel function for booking hotel.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: E define method wey dey book hotel with parameters and return type annotated.

The method go make dictionary of parameters for hotel booking request and send GET request go SERP API. E go check response status, if e successful e go return hotel properties, if no, e go return None.

Flight Booking Method: 

- `@kernel_function(description="booking flight")`: Na decorator wey talk say dis function na kernel function for booking flight.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: E define method wey dey book flight with annotated parameters and return type.

The method go make dictionaries of parameters for outbound and return flight requests, then send GET requests to SERP API. E go check response status and if e successful, e go add flight info to result string, if no, e go print error message. The method go return result string wey get flight info.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explanation:
Import Statements: Import necessary modules for Azure credentials, AI agent, chat message content, author role, and kernel function decorator.

Asynchronous Context Manager: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Dis dey set up asynchronous context manager to handle Azure credentials and create AI agent client.

Agent Name and Instructions: 
- `AGENT_NAME = "BookingAgent"`: E define di name of di agent.
- `AGENT_INSTRUCTIONS = """..."""`: E give detailed instructions for di agent how to handle booking requests.

Create Agent Definition: `agent_definition = await client.agents.create_agent(...)`: E create agent definition wit di specified model, name, and instructions.

Create AzureAI Agent: `agent = AzureAIAgent(...)`: E create AzureAI agent using di client, agent definition, and di defined plugin.

Create Thread: `thread: AzureAIAgentThread | None = None`: E create thread for di agent. E no mandatory to first create thread - if `None` value dey, new thread go create during di first invocation and e go return as part of di response.

User Inputs: `user_inputs = ["..."]`: E define list of user inputs for di agent to process.

For di finally block, delete di thread and agent to clean up resources.


# Authentication

Di `DefaultAzureCredential` klasse na part of di Azure SDK for Python. E dey provide one kain default way to authenticate wit Azure services. E dey try authenticate wit different methods for one correct order, like environment variables, managed identity, and Azure CLI credentials.

Asynchronous Operations: Di aio module dey show say di DefaultAzureCredential klasse support asynchronous operations. Dis mean say you fit use am wit asyncio to run authentication requests wey no go block di process.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis document na wetin AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator) translate. Even though we dey try make am correct, abeg sabi say machine translation fit get mistake or error. Original document for im own language na im be correct one. For important matter, make you use professional human translation. We no go take responsibility if anything no clear well or wrong understand happen because of this translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
